In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
features = pd.read_csv('temps.csv')

# See what the data look like
features.head()

In [ ]:
print('Data dimension:', features.shape)

In [ ]:
# handle time data
import datetime

# get year, month, day separately
years = features['year']
months = features['month']
days = features['day']

# datetime format
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

In [ ]:
dates[:5]

In [ ]:
# Prepare to draw
# Set the teme to default
plt.style.use('fivethirtyeight')

# Set the layout
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
fig.autofmt_xdate(rotation=45)

# Label values
ax1.plot(dates, features['actual'])
ax1.set_xlabel(''); ax1.set_ylabel('Temperature'); ax1.set_title('Max Temp')

# Yesterday
ax2.plot(dates, features['temp_1'])
ax2.set_xlabel(''); ax2.set_ylabel('Temperature'); ax2.set_title('Previous Max Temp')

# The day before yesterday
ax3.plot(dates, features['temp_2'])
ax3.set_xlabel('Date'); ax3.set_ylabel('Temperature'); ax3.set_title('Two Days Prior Max Temp')

# My silly bro
ax4.plot(dates, features['friend'])
ax4.set_xlabel('Date'); ax4.set_ylabel('Temperature'); ax4.set_title('Friend Estimate')

plt.tight_layout(pad=2)
plt.show()

In [ ]:
# Dummy encodings
features = pd.get_dummies(features)
features.head(5)

In [ ]:
# Labels
labels = np.array(features['actual'])

# Remove labels from features
features = features.drop('actual', axis=1)

# Store the names independently to avoid possible dangers
feature_list = list(features.columns)

# Convert into suitable formats
features = np.array(features)

In [ ]:
features.shape

In [ ]:
from sklearn import preprocessing
input_features = preprocessing.StandardScaler().fit_transform(features)

In [ ]:
input_features[0]

In [ ]:
x = torch.tensor(input_features, dtype=torch.float)
y = torch.tensor(labels, dtype=torch.float)

# Initialise the weight arguments
weights = torch.randn((14, 128), dtype=torch.float, requires_grad=True)
biases = torch.randn(128, dtype=torch.float, requires_grad=True)
weights2 = torch.randn((128, 1), dtype=torch.float, requires_grad=True)
biases2 = torch.randn(1, dtype=torch.float, requires_grad=True)

learning_rate = 0.001
losses = []

for i in range(1000):
    # Calculate the hidden layers
    hidden = x.mm(weights) + biases
    # Include activating functions
    hidden = torch.relu(hidden)
    # Predict the result
    predictions = hidden.mm(weights2) + biases2
    # Calculate the losses
    loss = torch.mean((predictions - y) ** 2)
    losses.append(loss.data.numpy())
    
    # Print the loss value
    if i % 100 == 0:
        print('loss:', loss)
    # Backward propagating calculation
    loss.backward()
    
    # Update arguments
    weights.data.add_(-learning_rate * weights.grad.data)
    biases.data.add_(-learning_rate * biases.grad.data)
    weights2.data.add_(-learning_rate * weights2.grad.data)
    biases2.data.add_(-learning_rate * biases2.grad.data)
    
    # Remember to do empty every iteration
    weights.grad.data.zero_()
    biases.grad.data.zero_()
    weights2.grad.data.zero_()
    biases2.grad.data.zero_()

In [ ]:
predictions.shape

In [ ]:
input_size = input_features.shape[1]
hidden_size = 128
output_size = 1
batch_size = 16
my_nn = torch.nn.Sequential(
    torch.nn.Linear(input_size, hidden_size),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_size, output_size)
)
cost = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(my_nn.parameters(), lr=0.001)

In [ ]:
# Train the network
losses = []
for i in range(1000):
    batch_loss = []
    # Train using MINI-Batch method
    for start in range(0, len(input_features), batch_size):
        end = start + batch_size if start + batch_size < len(input_features) else len(input_features)
        xx = torch.tensor(input_features[start:end], dtype=torch.float, requires_grad=True)
        yy = torch.tensor(labels[start:end], dtype=torch.float, requires_grad=True)
        prediction = my_nn(xx)
        loss = cost(prediction, yy)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        batch_loss.append(loss.data.numpy())
        
    # Print the losses
    if i % 100 == 0:
        losses.append(np.mean(batch_loss))
        print(i, np.mean(batch_loss))

In [ ]:
x = torch.tensor(input_features, dtype=torch.float)
predict = my_nn(x).data.numpy()

In [ ]:
# Convert datetime format
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# Create a table to store label values corresponding to their dates
true_data = pd.DataFrame(data={'date': dates, 'actual': labels})

# Similarly, create one to store model predictions corresponding to their dates
months = features[:, feature_list.index('month')]
days = features[:, feature_list.index('day')]
years = features[:, feature_list.index('year')]

test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in test_dates]
predictions_data = pd.DataFrame(data={'date': test_dates, 'prediction': predict.reshape(-1)})

In [ ]:
# Real values
plt.plot(true_data['date'], true_data['actual'], 'b-', label='actual')

# Predictions
plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label='prediction')
plt.xticks(rotation=60)
plt.legend()

# Plot name
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual and Predicted Values')
plt.show()